In [1]:
%load_ext autoreload
%autoreload 2

import copy, os, socket, sys, time
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs import data, fl, nn, wandb
from libs.distributed import *

In [2]:
class FedArgs():
    def __init__(self):
        self.name = "client-3"
        self.num_clients = 50
        self.epochs = 51
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 64
        self.learning_rate = 0.001
        self.weight_decay = None
        self.cuda = False
        self.seed = 1
        self.topic = "VJH_020_2"
        self.broker_ip = '172.16.26.40:9092'
        self.schema_ip = 'http://172.16.26.40:8081'
        self.wait_to_consume = 10
        self.dataset = "lemon"
        self.model = nn.LemonNet()
        self.train_func = fl.train_model
        self.eval_func = fl.evaluate
        
fedargs = FedArgs()

In [3]:
project = 'fl-kafka-client'
name = 'VJH_020_1-' + fedargs.name
wb = wandb.init(name, project)
fedargs.num_clients = 1

wandb: Currently logged in as: kasyah (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [5]:
host = socket.gethostname()
clients = [host + ": " + fedargs.name]
dt = Distributed(clients, fedargs.broker_ip, fedargs.schema_ip, fedargs.wait_to_consume)

In [6]:
# Initialize Global and Client models
global_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_loader, test_loader = data.load_dataset(fedargs.dataset, fedargs.client_batch_size, fedargs.test_batch_size)

client_details = {"name": clients[0],
                  "train_loader": train_loader,
                  "test_loader": test_loader,
                  "model": copy.deepcopy(global_model),
                  "model_update": None}

In [7]:
def process(client, epoch, dt, model, train_loader, test_loader, fedargs, device):
    # Consume Models
    client_model_updates = dt.consume_model(client, fedargs.topic, model, epoch)
    if client in client_model_updates:
        client_model_updates.pop(client)
    print("Epoch: {}, Processing Client {}, Received {} Updates From {}".format(epoch, client, 
                                                                                len(client_model_updates), 
                                                                                list(client_model_updates.keys())))

    #if len(client_model_updates) != 0:
    #    model = fl.federated_avg(client_model_updates)
    
    # Train
    model_update, model, loss = fedargs.train_func(model, train_loader, 
                                                   fedargs.learning_rate,
                                                   fedargs.weight_decay,
                                                   fedargs.local_rounds, device)
    
    # Publish Model
    epoch = epoch + 1
    dt.produce_model(client, fedargs.topic, model, epoch)

    # Test, Plot and Log
    test_output = fedargs.eval_func(model, test_loader, device)
    print("Epoch: {}, Accuracy: {}, Test Loss: {}".format(epoch, test_output["accuracy"], test_output["test_loss"]))
    wb.log({client: {"epoch": epoch, "time": time.time(), "acc": test_output["accuracy"], "loss": test_output["test_loss"]}})

    return model

In [ ]:
# Federated Training
for epoch in tqdm(range(fedargs.epochs)):
    print("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    client_details['model'] = process(client_details['name'], epoch, dt,
                                              client_details['model'],
                                              client_details['train_loader'],
                                              client_details['test_loader'],
                                              fedargs, device)

  0%|          | 0/51 [00:00<?, ?it/s]

Federated Training Epoch 0 of 51
Epoch: 0, Processing Client bladecluster.iitp.org: client-3, Received 0 Updates From []
Producing user records to topic VJH_020_2. ^C to exit.
Flushing records...
User record b'bladecluster.iitp.org: client-3' successfully produced to VJH_020_2 [0] at offset 713


  2%|▏         | 1/51 [00:29<24:49, 29.79s/it]

Epoch: 1, Accuracy: 16.0, Test Loss: 2.1622895050048827
Federated Training Epoch 1 of 51
Epoch: 1, Processing Client bladecluster.iitp.org: client-3, Received 1 Updates From ['bladecluster.iitp.org: client-2']
Producing user records to topic VJH_020_2. ^C to exit.
Flushing records...
User record b'bladecluster.iitp.org: client-3' successfully produced to VJH_020_2 [0] at offset 716


  4%|▍         | 2/51 [00:56<22:43, 27.82s/it]

...

Epoch: 30, Accuracy: 72.0, Test Loss: 1.5688754653930663
Federated Training Epoch 30 of 51
Epoch: 30, Processing Client bladecluster.iitp.org: client-3, Received 1 Updates From ['bladecluster.iitp.org: client-2']
Producing user records to topic VJH_020_2. ^C to exit.
Flushing records...
User record b'bladecluster.iitp.org: client-3' successfully produced to VJH_020_2 [0] at offset 776


 61%|██████    | 31/51 [13:42<08:48, 26.44s/it]

Epoch: 31, Accuracy: 76.0, Test Loss: 1.3092716217041016
Federated Training Epoch 31 of 51
Epoch: 31, Processing Client bladecluster.iitp.org: client-3, Received 1 Updates From ['bladecluster.iitp.org: client-2']
Producing user records to topic VJH_020_2. ^C to exit.
Flushing records...
User record b'bladecluster.iitp.org: client-3' successfully produced to VJH_020_2 [0] at offset 778


 63%|██████▎   | 32/51 [14:08<08:22, 26.47s/it]

Epoch: 32, Accuracy: 75.0, Test Loss: 1.6535157775878906
Federated Training Epoch 32 of 51
